## 1. Simulate Transaction Data

In [ ]:
import random
import pandas as pd

random.seed(42)
items_pool = ['Indomie', 'Bread', 'Juice', 'Eggs', 'Oatmeal', 'Onions', 'Tomatoes', 'Potatoes']

transactions = []
for _ in range(50): # Generate 50 transactions
    transaction = set(random.sample(items_pool, k=random.randint(2, 5)))
    # Boost some common pairs
    if random.random() < 0.3:
        transaction.update(['Bread', 'Eggs'])
    if random.random() < 0.2:
        transaction.update(['Juice', 'Oatmeal'])
    transactions.append(list(transaction))

## 2. Analyze with Apriori 

In [24]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

# One-hot encode the transaction data
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df = pd.DataFrame(te_ary, columns=te.columns_)

# Apply Apriori algorithm with minimum support of 0.3 (30%)
frequent_itemsets = apriori(df, min_support=0.3, use_colnames=True)
print(frequent_itemsets)

    support              itemsets
0      0.66               (Bread)
1      0.58                (Eggs)
2      0.42             (Indomie)
3      0.58               (Juice)
4      0.48             (Oatmeal)
5      0.52              (Onions)
6      0.38            (Potatoes)
7      0.42            (Tomatoes)
8      0.50         (Eggs, Bread)
9      0.40        (Bread, Juice)
10     0.30      (Bread, Oatmeal)
11     0.32       (Onions, Bread)
12     0.30     (Bread, Tomatoes)
13     0.34         (Eggs, Juice)
14     0.32      (Juice, Oatmeal)
15     0.30  (Eggs, Bread, Juice)


## Step 3: Association Rules

The following rules were found with minimum support 0.3 and minimum confidence 0.7:

| Antecedents | Consequents | Support | Confidence | Lift |
|-------------|-------------|---------|------------|------|
| (Eggs)      | (Bread)     | 0.5     | 0.86       | 1.31 |
| (Bread)     | (Eggs)      | 0.5     | 0.76       | 1.31 |
| (Tomatoes)  | (Bread)     | 0.3     | 0.71       | 1.08 |

**Example interpretation:**  
If a customer buys Eggs, they are also likely to buy Bread (confidence: 0.86).  
This means these items are often bought together, so a store could put them on the same shelf or offer a combo deal.

In [ ]:
from mlxtend.frequent_patterns import association_rules

rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7)

# Show rules
if not rules.empty:
    print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].head(3))
else:
    print("No rules found. Try increasing transactions or lowering confidence threshold.")


  antecedents consequents  support  confidence      lift
0      (Eggs)     (Bread)      0.5    0.862069  1.306165
1     (Bread)      (Eggs)      0.5    0.757576  1.306165
2  (Tomatoes)     (Bread)      0.3    0.714286  1.082251
